In [1]:
import java.nio.file.Files
import java.nio.file.Path

data class BenchmarkData(val name: String, val score: Double, val error: Double) {
    override fun toString(): String = "$name: $score ± $error ns/op"
}

fun readHotspotBenchmarkData(benchmarkName: String) = Files.readAllLines(Path.of("build", "results", "jmh", "results.txt"))
    .mapNotNull {
        val r = Regex("$benchmarkName\\.([^ ]+) +avgt +\\d+ +([0-9.]+) +± +([0-9.]+) +ns/op")
        val match = r.matchEntire(it) ?: return@mapNotNull null
        BenchmarkData(
            match.groups[1]!!.value,
            match.groups[2]!!.value.toDouble(),
            match.groups[3]!!.value.toDouble(),
        )
    }
(readHotspotBenchmarkData("Wolf3dBenchmark") + readHotspotBenchmarkData("FibonacciBenchmark")).joinToString("\n")

baselineDouble: 917835.788 ± 2297.417 ns/op
baselineFloat: 890312.969 ± 1636.437 ns/op
inlineDouble: 916968.087 ± 2338.012 ns/op
inlineFloat: 898776.711 ± 2532.563 ns/op
longPackFloat: 1591993.034 ± 683.825 ns/op
mutableRefDouble: 644314.935 ± 2209.507 ns/op
mutableRefFloatIn1Long: 1551868.621 ± 552.106 ns/op
mutableRefFloatIn2Longs: 643496.128 ± 936.294 ns/op
valueDouble: 670610.691 ± 886.765 ns/op
valueFloat: 665058.83 ± 7302.755 ns/op
valueInlineDouble: 645546.683 ± 800.916 ns/op
valueInlineFloat: 672885.812 ± 8046.276 ns/op
valuePreserveBoxDouble: 644801.406 ± 684.261 ns/op
valuePreserveBoxFloat: 667967.836 ± 9624.674 ns/op
baselineInt: 379.562 ± 2.372 ns/op
baselineLong: 403.359 ± 3.133 ns/op
longPackInt: 188.212 ± 0.541 ns/op
mutableRefIntIn1Long: 299.232 ± 0.773 ns/op
mutableRefIntIn2Longs: 249.907 ± 1.374 ns/op
mutableRefLong: 238.002 ± 1.016 ns/op
valueInt: 357.104 ± 0.745 ns/op
valueLong: 360.018 ± 1.989 ns/op
valuePreserveBoxInt: 356.658 ± 0.522 ns/op
valuePreserveBoxLong: 3

In [2]:
%use lets-plot

In [3]:
import org.jetbrains.letsPlot.intern.PosKind
import org.jetbrains.letsPlot.intern.StatKind
import org.jetbrains.letsPlot.intern.layer.PosOptions
import org.jetbrains.letsPlot.intern.layer.StatOptions

fun makePlot(name: String, platform: String, benchmarkLines: List<BenchmarkData>) {
    if (benchmarkLines.isEmpty()) return
    val names = benchmarkLines.map { it.name }
    val scores = benchmarkLines.map { it.score }
    val scoresMinusErrors = benchmarkLines.map { it.score - it.error / 2 }
    val scoresPlusErrors = benchmarkLines.map { it.score + it.error / 2 }
    val baseline = listOf("Float", "Double", "Int", "Long").mapNotNull { type ->
        val result = benchmarkLines.singleOrNull { it.name == "baseline$type" }
        if (result != null) type to result else null
    }.toMap()
    val maxScore = scoresPlusErrors.max()
    val plot = letsPlot() +
            labs("$name ($platform)", fill = "Benchmark", y = "ns/op") +
            geomBar(stat = StatOptions(kind = StatKind.IDENTITY)) {
                x = names
                y = scores
                fill = names
            } +
            geomErrorBar(
                width = 0.5,
                position = PosOptions(kind = PosKind.DODGE),
                size = 0.6
            ) {
                x = names
                ymin = scoresMinusErrors
                ymax = scoresPlusErrors
            } +
            geomText(
                stat = StatOptions(kind = StatKind.IDENTITY),
                position = positionStack(vjust = 1.0),
                size = 5,
            ) {
                x = names
                y = scores.map {
                    val threshold = 0.08
                    if (it < maxScore * threshold) it + threshold * maxScore / 2 else it / 2
                }
                label = benchmarkLines.map {
                    val type = when {
                        it.name.contains("Float") -> "Float"
                        it.name.contains("Int") -> "Int"
                        it.name.contains("Double") -> "Double"
                        it.name.contains("Long") -> "Long"
                        else -> error("Unknown name: ${it.name}")
                    }
                    val score = it.score / baseline[type]!!.score
                    val error = it.error / baseline[type]!!.score
                    "${String.format("%.0f", score * 100)}%\n±\n${String.format("%.1f", error * 100)}%\n${type}"
                }
            } +
            scaleXDiscrete(
                labels = names,
                name = "",
                limits = names + listOf(""),
                breaks = names.toList()
            ) +
            scaleYContinuous(limits = 0 to maxScore, name = "ns/op") +
            theme(plotTitle = elementText(hjust = 0.5)).legendPositionRight() +
            ggsize(1000, 800)
    ggsave(plot = plot, filename = "$name ($platform).svg")
    plot.show()
}


In [4]:
makePlot("Wolf3d", "Hotspot", readHotspotBenchmarkData("Wolf3dBenchmark"))
makePlot("Fibonacci", "Hotspot", readHotspotBenchmarkData("FibonacciBenchmark"))

In [5]:
import java.io.FileFilter
import kotlin.io.path.listDirectoryEntries

fun readArtBenchmarkData(benchmarkName: String) = Path.of("android-benchmark/build/outputs/androidTest-results/connected")
    .toFile().listFiles(FileFilter { it.isDirectory })!!.single()
    .listFiles { dir, name -> name.startsWith("logcat-org.jetbrains.") }!!.flatMap { it.readLines() }
    .mapNotNull {
        val pattern = Regex("^\\d{2}-\\d{2} +\\d+:\\d+:\\d+.\\d+ +\\d+ +\\d+ +I +Benchmark: +$benchmarkName\\.([^\\[]+)\\[Metric \\(timeNs\\) +results: +median +([^,]+), +min [^,]+, +max [^,]+, +standardDeviation: +([^,]+), +.*")
        val match = pattern.matchEntire(it) ?: return@mapNotNull null
        BenchmarkData(
            match.groups[1]!!.value,
            match.groups[2]!!.value.toDouble(),
            match.groups[3]!!.value.toDouble(),
        )
     }.let { lines ->
         val indexes = readHotspotBenchmarkData(benchmarkName).mapIndexed { index, benchmarkData -> benchmarkData.name to index }.toMap()
         if (lines.all { it.name in indexes }) lines.sortedBy { indexes[it.name]!! } else lines
     }
(readArtBenchmarkData("Wolf3dBenchmark") + readArtBenchmarkData("FibonacciBenchmark")).joinToString("\n")

baselineDouble: 1.13492355625E7 ± 567324.6828501407 ns/op
baselineFloat: 1.0749055611111112E7 ± 208081.14501088014 ns/op
inlineDouble: 2.6348384666666668E7 ± 897765.9408843282 ns/op
inlineFloat: 1.81996192E7 ± 869342.0280970045 ns/op
longPackFloat: 4498985.159090909 ± 6425.947585356221 ns/op
mutableRefDouble: 3495787.7857142854 ± 77551.18177884219 ns/op
mutableRefFloatIn1Long: 5392803.416666666 ± 3577.281170979714 ns/op
mutableRefFloatIn2Longs: 3790135.3653846155 ± 4171.6093971590835 ns/op
valueDouble: 1.3040708785714287E7 ± 560399.4740374224 ns/op
valueFloat: 9895196.1 ± 216034.87733118472 ns/op
valueInlineDouble: 553418.2936507936 ± 9597.880179356167 ns/op
valueInlineFloat: 668067.6308724831 ± 4397.283857968942 ns/op
valuePreserveBoxDouble: 1.12226058125E7 ± 567258.6931291434 ns/op
valuePreserveBoxFloat: 7904216.375 ± 322249.39055868314 ns/op
baselineInt: 2823.7224371373304 ± 31.95413745546588 ns/op
baselineLong: 2063.303395225464 ± 169.45488866622892 ns/op
longPackInt: 175.090189280

In [6]:
makePlot("Wolf3d", "ART", readArtBenchmarkData("Wolf3dBenchmark"))
makePlot("Fibonacci", "ART", readArtBenchmarkData("FibonacciBenchmark"))